In [0]:
import numpy
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras.optimizers import SGD
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [0]:
# fix random seed for reproducibility
x = 7
numpy.random.seed(x)
x

7

In [0]:
# load dataset
import io
from google.colab import files
uploaded = files.upload()
dataframe = pd.read_csv(io.StringIO(uploaded['sonar.csv'].decode('utf-8')))
dataframe

Saving sonar.csv to sonar (1).csv


,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
0,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
1,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
2,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0.3973,0.2741,0.3690,0.5556,0.4846,0.3140,0.5334,0.5256,0.2520,0.2090,0.3559,0.6260,0.7340,0.6120,0.3497,0.3953,0.3012,0.5408,0.8814,0.9857,0.9167,0.6121,0.5006,0.3210,0.3202,0.4295,0.3654,0.2655,0.1576,0.0681,0.0294,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
3,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0.4636,0.4148,0.4292,0.5730,0.5399,0.3161,0.2285,0.6995,1.0000,0.7262,0.4724,0.5103,0.5459,0.2881,0.0981,0.1951,0.4181,0.4604,0.3217,0.2828,0.2430,0.1979,0.2444,0.1847,0.0841,0.0692,0.0528,0.0357,0.0085,0.0230,0.0046,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
4,0.0286,0.0453,0.0277,0.0174,0.0384,0.0990,0.1201,0.1833,0.2105,0.3039,0.2988,0.4250,0.6343,0.8198,1.0000,0.9988,0.9508,0.9025,0.7234,0.5122,0.2074,0.3985,0.5890,0.2872,0.2043,0.5782,0.5389,0.3750,0.3411,0.5067,0.5580,0.4778,0.3299,0.2198,0.1407,0.2856,0.3807,0.4158,0.4054,0.3296,0.2707,0.2650,0.0723,0.1238,0.1192,0.1089,0.0623,0.0494,0.0264,0.0081,0.0104,0.0045,0.0014,0.0038,0.0013,0.0089,0.0057,0.0027,0.0051,0.0062,R
5,0.0317,0.0956,0.1321,0.1408,0.1674,0.1710,0.0731,0.1401,0.2083,0.3513,0.1786,0.0658,0.0513,0.3752,0.5419,0.5440,0.5150,0.4262,0.2024,0.4233,0.7723,0.9735,0.9390,0.5559,0.5268,0.6826,0.5713,0.5429,0.2177,0.2149,0.5811,0.6323,0.2965,0.1873,0.2969,0.5163,0.6153,0.4283,0.5479,0.6133,0.5017,0.2377,0.1957,0.1749,0.1304,0.0597,0.1124,0.1047,0.0507,0.0159,0.0195,0.0201,0.0248,0.0131,0.0070,0.0138,0.0092,0.0143,0.0036,0.0103,R
6,0.0519,0.0548,0.0842,0.0319,0.1158,0.0922,0.1027,0.0613,0.1465,0.2838,0.2802,0.3086,0.2657,0.3801,0.5626,0.4376,0.2617,0.1199,0.6676,0.9402,0.7832,0.5352,0.6809,0.9174,0.7613,0.8220,0.8872,0.6091,0.2967,0.1103,0.1318,0.0624,0.0990,0.4006,0.3666,0.1050,0.1915,0.3930,0.4288,0.2546,0.1151,0.2196,0.1879,0.1437,0.2146,0.2360,0.1125,0.0254,0.0285,0.0178,0.0052,0.0081,0.0120,0.0045,0.0121,0.0097,0.0085,0.0047,0.0048,0.0053,R
7,0.0223,0.0375,0.0484,0.0475,0.0647,0.0591,0.0753,0.0098,0.0684,0.1487,0.1156,0.1654,0.3833,0.3598,0.1713,0.1136,0.0349,0.3796,0.7401,0.9925,0.9802,0.8890,0.6712,0.4286,0.3374,0.7366,0.9611,0.7353,0.4856,0.1594,0.3007,0.4096,0.3170,0.3305,0.3408,0.2186,0.2463,0.2726,0.1680,0.2792,0.2558,0.1740,0.2121,0.1099,0.0985,0.1271,0.1459,0.1164,0.0777,0.0439,0.0061,0.0145,0.0128,0.0145,0.0058,0.0049,0.0065,0.0093,0.0059,0.0022,R
8,0.0164,0.0173,0.0347,0.0070,0.0187,0.0671,0.1056,0.0697,0.0962,0.0251,0.0801,0.1056,0.1266,0.0890,0.0198,0.1133,0.2826,0.3234,0.3238,0.4333,0.6068,0.7652,0.9203,0.9719,0.9207,0.7545,0

In [0]:
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)


In [0]:
Y

array(['R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M

In [0]:
X

array([[0.0453, 0.0523, 0.0843, ..., 0.0049, 0.0052, 0.0044],
       [0.0262, 0.0582, 0.1099, ..., 0.0164, 0.0095, 0.0078],
       [0.01  , 0.0171, 0.0623, ..., 0.0044, 0.004 , 0.0117],
       ...,
       [0.0522, 0.0437, 0.018 , ..., 0.0138, 0.0077, 0.0031],
       [0.0303, 0.0353, 0.049 , ..., 0.0079, 0.0036, 0.0048],
       [0.026 , 0.0363, 0.0136, ..., 0.0036, 0.0061, 0.0115]])

In [0]:
# baseline
def create_baseline():
	# create model
	model = Sequential()
	model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
	model.add(Dense(30, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
	# Compile model
	sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
	model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
	return model

np.random.seed(x)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn = create_baseline, epochs = 300, batch_size = 16, verbose = 0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = x)
results = cross_val_score(pipeline, X, encoded_Y, cv = kfold)
print('Baseline : %.2f%% (%.2f % %)' % (results.mean()* 100, results.std()* 100))

W0826 10:19:17.531154 139785231251328 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0826 10:19:17.580011 139785231251328 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0826 10:19:17.587827 139785231251328 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0826 10:19:17.647001 139785231251328 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0826 10:19:17.655690 139785231251328 deprecation_wrapper.

Baseline : 84.04% (5.34 %)


In [0]:
#Dropout the input layer with weight constraint
def Dropout_visible_model():
  model = Sequential()
  model.add(Dense(60, input_dim = 60, kernel_initializer = 'normal', activation = 'relu'))
  model.add(Dropout(0.2))
  model.add(Dense(30, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  
  # Compile model
  sgd = SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=False)
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

np.random.seed(x)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn = Dropout_visible_model, epochs = 300, batch_size = 16, verbose = 0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = x)
results = cross_val_score(pipeline, X, encoded_Y, cv = kfold)
print('Dropout visible model : %.2f%% (%.2f % %)' % (results.mean()* 100, results.std()* 100))

Dropout visible model : 84.54% (6.38 %)


In [0]:
#Improve performance by increasing number of epochs.
def Dropout_visible_model_epochs():
  model = Sequential()
  model.add(Dense(60, input_dim = 60, kernel_initializer = 'normal', activation = 'relu'))
  model.add(Dropout(0.2))
  model.add(Dense(30, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  
  # Compile model
  sgd = SGD(lr=0.02, momentum=0.9, decay=0.0, nesterov=False)
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

np.random.seed(x)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn = Dropout_visible_model_epochs, epochs = 500, batch_size = 16, verbose = 0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = x)
results = cross_val_score(pipeline, X, encoded_Y, cv = kfold)
print('Dropout visible model : %.2f%% (%.2f % %)' % (results.mean()* 100, results.std()* 100))

Dropout visible model : 82.11% (6.08 %)


In [0]:
#Dropout the input layer with weight constraint
def Dropout_hidden_layer():
  model = Sequential()
  model.add(Dense(60, input_dim = 60, kernel_initializer = 'normal', activation = 'relu'))
  model.add(Dense(30, kernel_initializerbn='normal', activation='relu'))
  model.add(Dropout(0.02))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  
  # Compile model
  sgd = SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=False)
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

np.random.seed(x)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn = Dropout_hidden_layer, epochs = 300, batch_size = 16, verbose = 0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = x)
results = cross_val_score(pipeline, X, encoded_Y, cv = kfold)
print('Dropout hidden model : %.2f%% (%.2f % %)' % (results.mean()* 100, results.std()* 100))

Dropout hidden model : 83.11% (5.71 %)


In [0]:
#Improve performance by increase number of epochs.
def Dropout_hidden_layer_epochs():
  model = Sequential()
  model.add(Dense(60, input_dim = 60, kernel_initializer = 'normal', activation = 'relu'))
  model.add(Dense(30, kernel_initializer='normal', activation='relu'))
  model.add(Dropout(0.02))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  
  # Compile model
  sgd = SGD(lr=0.02, momentum=0.9, decay=0.0, nesterov=False)
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

np.random.seed(x)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn = Dropout_hidden_layer_epochs, epochs = 500, batch_size = 16, verbose = 0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = x)
results = cross_val_score(pipeline, X, encoded_Y, cv = kfold)
print('Dropout hidden model : %.2f%% (%.2f % %)' % (results.mean()* 100, results.std()* 100))

Dropout hidden model : 83.54% (5.02 %)


In [0]:
#Try different drop out values
def Dropout_values_layer():
  model = Sequential()
  model.add(Dense(60, input_dim = 60, kernel_initializer = 'normal', activation = 'relu'))
  model.add(Dropout(0.04))
  model.add(Dense(30, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  
  # Compile model
  sgd = SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=False)
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

np.random.seed(x)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn = Dropout_values_layer, epochs = 300, batch_size = 16, verbose = 0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = x)
results = cross_val_score(pipeline, X, encoded_Y, cv = kfold)
print('Dropout hidden model : %.2f%% (%.2f % %)' % (results.mean()* 100, results.std()* 100))

Dropout hidden model : 84.56% (4.09 %)


In [0]:
#Try using a larger network.
def Dropout_Larger_model():
  model = Sequential()
  model.add(Dense(60, input_dim = 60, kernel_initializer = 'normal', activation = 'relu'))
  model.add(Dropout(0.02))
  model.add(Dense(60, kernel_initializer='normal', activation='relu'))
  model.add(Dense(60, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  
  # Compile model
  sgd = SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=False)
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

np.random.seed(x)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn = Dropout_Larger_model, epochs = 300, batch_size = 16, verbose = 0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = x)
results = cross_val_score(pipeline, X, encoded_Y, cv = kfold)
print('Dropout Larger model : %.2f%% (%.2f % %)' % (results.mean()* 100, results.std()* 100))

Dropout Larger model : 85.06% (3.80 %)


In [0]:
#Using Dropouts on both hidden and visible layers
def Dropout_visible_hidden():
  model = Sequential()
  model.add(Dense(60, input_dim = 60, kernel_initializer = 'normal', activation = 'relu'))
  model.add(Dropout(0.02))
  model.add(Dense(30, kernel_initializer='normal', activation='relu'))
  model.add(Dropout(0.02))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  
  # Compile model
  sgd = SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=False)
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

np.random.seed(x)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn = Dropout_visible_hidden, epochs = 300, batch_size = 16, verbose = 0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = x)
results = cross_val_score(pipeline, X, encoded_Y, cv = kfold)
print('Dropout visible hidden model : %.2f%% (%.2f % %)' % (results.mean()* 100, results.std()* 100))

Dropout visible hidden model : 84.56% (4.61 %)


In [0]:
#Large learning rates with decay and larger momentum
def Learning_momentum_decay():
  model = Sequential()
  model.add(Dense(60, input_dim = 60, kernel_initializer = 'normal', activation = 'relu'))
  model.add(Dropout(0.02))
  model.add(Dense(30, kernel_initializer='normal', activation='relu'))
  model.add(Dropout(0.02))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
  
  # Compile model
  sgd = SGD(lr=0.04, momentum=0.9, decay=0.2, nesterov=False)
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

np.random.seed(x)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn = Learning_momentum_decay, epochs = 300, batch_size = 16, verbose = 0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = x)
results = cross_val_score(pipeline, X, encoded_Y, cv = kfold)
print('Learning momentum decay model : %.2f%% (%.2f % %)' % (results.mean()* 100, results.std()* 100))

Learning momentum decay model : 79.59% (6.01 %)


In [0]:
#Constraining the size of the network weights
def max_norm_model():
  model = Sequential()
  model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu', kernel_constraint=maxnorm(5)))
  model.add(Dense(30, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

  # Compile model
  sgd = SGD(lr=0.04, momentum=0.9, decay=0.2, nesterov=False)
  model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

np.random.seed(x)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn = max_norm_model, epochs = 300, batch_size = 16, verbose = 0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = x)
results = cross_val_score(pipeline, X, encoded_Y, cv = kfold)
print('Dropout Max norm model : %.2f%% (%.2f % %)' % (results.mean()* 100, results.std()* 100))

Dropout Max norm model : 75.75% (4.72 %)
